In [1]:
import re
import spacy
import tweepy
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from spacy.tokens import Doc
from spacy.tokens import DocBin
from spacy.training import Example
from tweepy import Stream, TweepError
from datetime import datetime, timedelta
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report

Carregando base com pandas.

In [34]:
data = pd.read_csv('base_sentimentos_twitter.csv')

Limpando texto: apagando tabs, quebra de linha e múltiplos espaços em branco seguidos, apagando citações a usuários (@), e apagando links. Outras etapas de pré-processamento estão implementadas na spacy.

In [3]:
def clean_text(text):
    if pd.isnull(text):
        return None
    return re.sub('( +)', " ", re.sub(r"( +)|([\t\n])|(@[a-zA-Z0-9_\-+\.:]+)|(http[a-zA-Z0-9/\-:&\.]+)", " ", text)).strip()

In [4]:
data['tweet'] = data['tweet'].apply(clean_text)

In [5]:
data

,tweet,sentiment
0,Impressionante essa garota. Nunca vi tanta mal...,-1
1,a sarinha não deixa eu estourar essa espinha d...,-1
2,Depois de quase dois meses de termo hj me bate...,-1
3,e vc amor?,-1
4,n da eu vejo qualquer coisa relacionada a pont...,-1
...,...,...
4397,Quase 10% da população e primeira dose só. Tá ...,-1
4398,pode? lá vai: Cachaceiro Cilva Cuba Caneta Cor...,-1
4399,Pegaria por dó,-1
4400,"Ah sim, Feliciano. Vc é bem honesto",-1


Dividindo a base aleatoriamente entre bases de treino (60%), validação (20%), e teste (20%) com scikit-learn.

In [6]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

In [7]:
train_data

,tweet,sentiment
2504,Amém 🙏 #oração #força #Gratidão #perdão #empat...,1
830,Minha namuie disse que eu sou chato 🥲,-1
3812,“O anjo mais velho” no fez o se emocionar e to...,1
4399,Pegaria por dó,-1
207,indo dormir,-1
...,...,...
1036,7.1 Primaveras🎈🎆 Agradeço a Deus por tudo e te...,1
1735,Simples assim! 😉 #Ternura #Empatia #Humanidade...,1
3848,“O anjo mais velho” no fez o se emocionar e to...,1
3385,queria receber demonstrações de #afeto e e #ca...,1


In [8]:
val_data

,tweet,sentiment
3626,“O anjo mais velho” no fez o se emocionar e to...,1
1572,#paz #harmonia #saude #amigos #familia #fé #sa...,1
604,"Sábado à noite em casa, no sofá....",-1
2666,#TodoPorAmor | A falta de amor ... |,1
2872,“O anjo mais velho” no fez o se emocionar e to...,1
...,...,...
702,Mentira... Te trato com maior amor e carinho a...,-1
1141,NF MUSIC Deseja a todos os Amigos e alunos uma...,1
333,Ex prefeito de Teresina Firmino Filho se suici...,-1
517,a dor de cotovelo pq a sarah foi eliminada e e...,-1


In [9]:
test_data

,tweet,sentiment
274,"Olá pessoal, esse ano não está muito fácil, qu...",-1
3431,Jesus = amor #jesus #catolicos #iglesia #enamo...,1
4021,Revendo o episódio final da quinta temporada d...,-1
1027,7.1 Primaveras🎈🎆 Agradeço a Deus por tudo e te...,1
1589,"Você não precisa desistir, só precisa descansa...",1
...,...,...
1032,7.1 Primaveras🎈🎆 Agradeço a Deus por tudo e te...,1
3281,💗🙏🏽Que o amor e a união se renove em todos nós...,1
527,Eu vendo que já se passou horas desde o nudes ...,-1
1200,#SputnikVGarantizaVida sábado,1


Carregando pipeline básico do spacy para português.

In [10]:
nlp=spacy.load('pt_core_news_lg')

Criando documentos com suas respectivas categorias (sentimentos): 1 - sentimento positivo, -1 - sentimento negativo. Nos termos dos problemas de classificação, os sentimentos são classes.

In [11]:
def make_docs(data):    
    docs = []    
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        doc.cats['1'] = label == 1
        doc.cats['-1'] = label == -1
        docs.append(doc)    
    return docs

In [12]:
docs_train = make_docs(train_data[['tweet', 'sentiment']].values)
docs_val = make_docs(val_data[['tweet', 'sentiment']].values)
docs_test = make_docs(test_data[['tweet', 'sentiment']].values)

Salvando as bases em arquivos binários para serem usadas para construção do modelo através da linha de comando.

In [13]:
doc_bin = DocBin(docs=docs_train)
doc_bin.to_disk("train.spacy")
doc_bin = DocBin(docs=docs_val)
doc_bin.to_disk("validation.spacy")
doc_bin = DocBin(docs=docs_test)
doc_bin.to_disk("test.spacy")

A partir deste ponto, deve-se seguir as instruções [neste link](https://spacy.io/usage/training). Após criar o arquivo base_config.cfg, executar a seguinte linha de comando no terminal, sem a exclamação (ou com a exclamação caso seja executado neste notebook). Este comando irá criar um arquivo config.cfg com todas as configurações necessárias para treinarmos um modelo. Os valores dos parâmetros são os valores padrão.

In [14]:
!python -m spacy init fill-config base_config.cfg config.cfg

2021-04-11 00:32:26.213411: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


O próximo comando inicializa o treinamento do modelo utilizando as bases de treino e validação geradas anteriormente, salvando o modelo em um diretório chamado 'output', utilizando a GPU disponível. Caso nenhuma GPU esteja disponível, remova a opção -gpu-id 0.

In [15]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./validation.spacy --output output --gpu-id 0

2021-04-11 00:32:33.461081: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-04-11 00:32:39,926] [INFO] Set up nlp object from config
[2021-04-11 00:32:39,955] [INFO] Pipeline: ['tok2vec', 'textcat']
[2021-04-11 00:32:39,963] [INFO] Created vocabulary
[2021-04-11 00:32:48,638] [INFO] Added vectors: pt_core_news_lg
[2021-04-11 00:32:48,638] [INFO] Finished initializing nlp object
[2021-04-11 00:32:58,486] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.06       49.37    0.49
  0     200    

Após treinar o modelo, podemos usá-lo na base de teste. Carregaremos o melhor modelo.

In [16]:
trained_nlp = spacy.load('output/model-best')

In [17]:
def predict(tokenizer, textcat, texts):
    docs = (tokenizer(text) for text in texts)
    return [pred for pred in textcat.pipe(texts)]

Aqui, testamos o modelo treinado na base de teste.

In [18]:
preds = predict(nlp.tokenizer, trained_nlp, test_data['tweet'])

In [19]:
predictions = []
for pred in preds:
    predictions.append(1 if pred.cats['1'] >= pred.cats['-1'] else -1)

In [20]:
print(classification_report(test_data['sentiment'], predictions))

              precision    recall  f1-score   support

          -1       0.82      0.89      0.85       264
           1       0.95      0.92      0.93       617

    accuracy                           0.91       881
   macro avg       0.88      0.90      0.89       881
weighted avg       0.91      0.91      0.91       881



In [37]:
teste_resultados_df = pd.DataFrame({'tweet': test_data['tweet'], 'prediction': predictions, 'sentiment': test_data['sentiment']})

Exemplos:

In [56]:
teste_resultados_df

,tweet,prediction,sentiment
274,"Olá pessoal, esse ano não está muito fácil, qu...",-1,-1
3431,Jesus = amor #jesus #catolicos #iglesia #enamo...,1,1
4021,Revendo o episódio final da quinta temporada d...,1,-1
1027,7.1 Primaveras🎈🎆 Agradeço a Deus por tudo e te...,1,1
1589,"Você não precisa desistir, só precisa descansa...",1,1
...,...,...,...
1032,7.1 Primaveras🎈🎆 Agradeço a Deus por tudo e te...,1,1
3281,💗🙏🏽Que o amor e a união se renove em todos nós...,1,1
527,Eu vendo que já se passou horas desde o nudes ...,-1,-1
1200,#SputnikVGarantizaVida sábado,1,1


In [39]:
teste_resultados_df.loc[274].values

array(['Olá pessoal, esse ano não está muito fácil, quem está falando nesse vídeo é o meu marido. Espero que você vejam. Obrigado pela compreensão. #ImpeachmentDeBolsonaroUrgente #obrigadopauloguedes #Deusacimadetudo #bbb21 #JulietteFreire #aceitodoaçoes',
       -1, -1], dtype=object)

In [40]:
teste_resultados_df.loc[3431].values

array(['Jesus = amor #jesus #catolicos #iglesia #enamorado', 1, 1],
      dtype=object)

In [55]:
teste_resultados_df.loc[4021].values

array(['Revendo o episódio final da quinta temporada de Game of Thrones enquanto espero o #BBB2021 começar. E fica a questão: onde vocês erraram ? Era tão bom! Da sexta temporada em diante ficou tudo cagado. #ranço #GoT #GameOfThrones',
       1, -1], dtype=object)

In [42]:
teste_resultados_df.loc[1027].values

array(['7.1 Primaveras🎈🎆 Agradeço a Deus por tudo e ter sempre os puros ao meu lado. Obg meu Deus por me permitir viver mais um ano de vida! 🎊💃🏽\U0001f973🎉🕺. . . . . . . . . #meus71anos #71primaveras #gratidão #aniversario',
       1, 1], dtype=object)